In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms, ToTensor
from torchvision.datasets import ImageFolder as IF
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
data_dir = "cifar10"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"
labels_file=data_dir + "/labels.txt"


In [ ]:
train_dataset = IF(train_dir, transform=ToTensor())
test_dataset = IF(test_dir, transform=ToTensor())

img,label = train_dataset[0]
img.shape, train_dataset.classes

In [ ]:
#create validation set
val_size = len(test_dataset) // 2
val_data, test_data = random_split(test_dataset, [val_size, len(test_dataset) - val_size])

batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader= DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader= DataLoader(test_data, batch_size=batch_size, shuffle=True)

len(train_loader), len(validation_loader), len(test_loader),

In [ ]:
#display some images
def display_images(images, labels):
    plt.figure(figsize=(20,16))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        ax.set_title(f"Label: {labels[i]}")
        plt.imshow(images[i].permute(1, 2, 0)) 
        plt.axis("off")
    plt.show()

for images, labels in train_loader:
    display_images(images, labels)
    break 


In [ ]:
class ClassifierBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)       #predictions
        loss = F.cross_entropy(out, labels) #loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                   
        loss = F.cross_entropy(out, labels) 
        acc = accuracy(out, labels)         
        return {'validation_loss': loss.detach(), 'validation_accuracy': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['validation_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  #combine losses
        batch_accs = [x['validation_accuracy'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()    #combine accuracies
        return {'validation_loss': epoch_loss.item(), 'validation_accuracy': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print(f"epoch {epoch+1}, train_loss: {result['train_loss']:.4f}, validation_loss: {result['validation_loss']:.4f}, validation_accuracy: {result['validation_accuracy']:.4f}")

       
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))



class ImageClassifier(ClassifierBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, xb):
        return self.network(xb)

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def train(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
       
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        #validation
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
model =ImageClassifier()
model

In [ ]:
epochs = 5
optimizer= torch.optim.Adam
lr = 0.001
history = train(epochs, lr, model, train_loader, validation_loader, optimizer)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['validation_accuracy'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('accuracy')
    
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['validation_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['training', 'validation'])
    plt.title('loss')
    
plot_losses(history)

In [ ]:
# testing on test data
def predict_image(img, model):
    # convert to a batch of 1
    xb =img.unsqueeze(0)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return train_dataset.classes[preds[0].item()]

In [ ]:
img, label = test_dataset[0]
plt.imshow(img.permute(1, 2, 0))
print('Label:', train_dataset.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
result = evaluate(model, test_loader)
result
